<h1 align="center">Youtube data construction</h1>
<h2 align=center> What did I watch on youtube ? </h2>
<br>
<center>The purpose of this script is to create differents dataframe containing informations about my youtube watch history.</center>
<center>A second notebook will be used to analyse this data.</center>

The purpose here is to : 
* fetch and clean the download my-youtube-watch-history received from google service, 
* web scraping of video details and channel infos from the Youtube API. 

In [2]:
import pandas as pd
from googleapiclient.discovery import build

<h2 align="center">Raw Data</h2>

In [4]:
raw_data = pd.read_json('watch-history.json')

In [6]:
raw_data.to_excel('raw_data.xlsx')

<h2 align="center">Cleaning of data</h2>

<h3> columns cleaning </h3>

In [8]:
wt = raw_data.copy()
wt = wt.drop(['header','products','activityControls','description'],axis=1)

In [10]:
wt.head(3)

,title,titleUrl,subtitles,time,details
0,Vous avez regardé Récupérer ses Données You...,https://www.youtube.com/watch?v=4Vo6kF4jWn8,"[{'name': 'Labo Des Réseaux', 'url': 'https://...",2024-03-31T14:22:02.374Z,NaN
1,Vous avez regardé FR_fr_1019_Spring_Instream_Y...,https://www.youtube.com/watch?v=JR9tdqcj0-4,NaN,2024-03-31T14:21:52.118Z,[{'name': 'Des annonces Google'}]
2,Vous avez regardé BeautifulSoup + Requests | W...,https://www.youtube.com/watch?v=bargNl2WeN4,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31T13:52:12.369Z,NaN


<h3> columns organization </h3>

In [12]:
#Cleaning Title
wt['title'] = wt['title'].str.replace('Vous avez regardé ','')
#Fetch video-ID from the url and place it on a different columns
wt['video_ID'] = wt['titleUrl'].str.split('=').str[1]

wt.head(3)

,title,titleUrl,subtitles,time,details,video_ID
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,"[{'name': 'Labo Des Réseaux', 'url': 'https://...",2024-03-31T14:22:02.374Z,NaN,4Vo6kF4jWn8
1,FR_fr_1019_Spring_Instream_YT_16x9_10_C_414081...,https://www.youtube.com/watch?v=JR9tdqcj0-4,NaN,2024-03-31T14:21:52.118Z,[{'name': 'Des annonces Google'}],JR9tdqcj0-4
2,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31T13:52:12.369Z,NaN,bargNl2WeN4


In [14]:
#Fetch the date and time in different columns and set them to datetime type
wt['time'] = wt['time'].str.replace('T',' ').str.split('.').str[0]
wt['time'] = wt['time'].str.replace('Z','')
wt['time'] = pd.to_datetime(wt['time'])

<h3>Filter the advertisement in a different dataframe</h3>

* Dataframe tableau_pub : only Google advertisement
* Dataframe working_table (wt) : extract the advertisement

In [16]:
tableau_pub = wt.copy()
tableau_pub = wt[wt['subtitles'].isnull()&wt['details'].notnull()]
tableau_pub = tableau_pub.drop(['video_ID'],axis=1)
tableau_pub.head(3)

,title,titleUrl,subtitles,time,details
1,FR_fr_1019_Spring_Instream_YT_16x9_10_C_414081...,https://www.youtube.com/watch?v=JR9tdqcj0-4,NaN,2024-03-31 14:21:52,[{'name': 'Des annonces Google'}]
3,02 ResponsableProjetCybersecuriteSI 16 9,https://www.youtube.com/watch?v=wKLFC8Uxvtc,NaN,2024-03-31 13:52:03,[{'name': 'Des annonces Google'}]
4,20221219 HELLOFRESH PEACE OF MIND V5 ssST,https://www.youtube.com/watch?v=xDlM2D1LGDQ,NaN,2024-03-31 13:51:40,[{'name': 'Des annonces Google'}]


In [18]:
wt = wt[wt['subtitles'].notnull()]
wt = wt.drop(['details'],axis=1)
wt.head()

,title,titleUrl,subtitles,time,video_ID
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,"[{'name': 'Labo Des Réseaux', 'url': 'https://...",2024-03-31 14:22:02,4Vo6kF4jWn8
2,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31 13:52:12,bargNl2WeN4
5,Inspecting Web Pages with HTML | Web Scraping ...,https://www.youtube.com/watch?v=q-kbzWjyPak,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31 13:43:46,q-kbzWjyPak
7,Building an Automated File Sorter in File Expl...,https://www.youtube.com/watch?v=gs0FNQR0njI,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31 13:39:53,gs0FNQR0njI
8,FREE Data Analyst Bootcamp!!,https://www.youtube.com/watch?v=rGx1QNdYzvs,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31 13:39:23,rGx1QNdYzvs


<h3> Fetch data from the subtitles columns </h3>

In [20]:
author = []
channel_url = []
for element in wt['subtitles']:
    author.append(element[0]["name"])
    channel_url.append(element[0]["url"])
wt['creator']=author
wt['channel_url']=channel_url

In [22]:
wt.head(3)

,title,titleUrl,subtitles,time,video_ID,creator,channel_url
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,"[{'name': 'Labo Des Réseaux', 'url': 'https://...",2024-03-31 14:22:02,4Vo6kF4jWn8,Labo Des Réseaux,https://www.youtube.com/channel/UCMJEpQVv0p3qz...
2,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31 13:52:12,bargNl2WeN4,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...
5,Inspecting Web Pages with HTML | Web Scraping ...,https://www.youtube.com/watch?v=q-kbzWjyPak,"[{'name': 'Alex The Analyst', 'url': 'https://...",2024-03-31 13:43:46,q-kbzWjyPak,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...


<h3> Index reset </h3>

In [24]:
wt.index=range(len(wt))
wt = wt.drop(['subtitles'],axis=1)
wt.head(3)

,title,titleUrl,time,video_ID,creator,channel_url
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,2024-03-31 14:22:02,4Vo6kF4jWn8,Labo Des Réseaux,https://www.youtube.com/channel/UCMJEpQVv0p3qz...
1,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,2024-03-31 13:52:12,bargNl2WeN4,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...
2,Inspecting Web Pages with HTML | Web Scraping ...,https://www.youtube.com/watch?v=q-kbzWjyPak,2024-03-31 13:43:46,q-kbzWjyPak,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...


<h3>Fetch channel_id from the channel_url </h3>

In [26]:
wt['channel_id']= wt['channel_url'].str.replace('https://www.youtube.com/channel/','')
wt.head(3)

,title,titleUrl,time,video_ID,creator,channel_url,channel_id
0,Récupérer ses Données YouTube,https://www.youtube.com/watch?v=4Vo6kF4jWn8,2024-03-31 14:22:02,4Vo6kF4jWn8,Labo Des Réseaux,https://www.youtube.com/channel/UCMJEpQVv0p3qz...,UCMJEpQVv0p3qzJnQ9IuiiMA
1,BeautifulSoup + Requests | Web Scraping in Python,https://www.youtube.com/watch?v=bargNl2WeN4,2024-03-31 13:52:12,bargNl2WeN4,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...,UC7cs8q-gJRlGwj4A8OmCmXg
2,Inspecting Web Pages with HTML | Web Scraping ...,https://www.youtube.com/watch?v=q-kbzWjyPak,2024-03-31 13:43:46,q-kbzWjyPak,Alex The Analyst,https://www.youtube.com/channel/UC7cs8q-gJRlGw...,UC7cs8q-gJRlGwj4A8OmCmXg


<h2 align=center> Recuperation channels data from Youtube API </h2>

* identification channels where I watched more than 10 videos
* convert there channel_id to a list
* pass the list to the youtube API and scrap their data

In [28]:
count = wt.channel_id.value_counts()
count = count[count > 10]
len(count)

312

In [30]:
channel_scraping = pd.DataFrame(data=count)
channel_scraping = channel_scraping.reset_index()

In [32]:
api_key ='***'
youtube= build('youtube', 'v3', developerKey=api_key)

In [34]:
channel_ids = channel_scraping['channel_id'].tolist()

In [36]:
def get_channel_stats(youtube, channel_ids):
    all_data = []
    
    for i in range(0,len(channel_ids),50):                  
        request = youtube.channels().list(
            part="snippet,contentDetails,statistics,id",
            id = ','.join(channel_ids[i:i+50]))
        response = request.execute()

        for i in range(len(response['items'])):
            data =dict(Channel_name = response['items'][i]['snippet']['title'],
                       channel_id = response['items'][i]['id'], 
                       Channel_date = response['items'][i]['snippet']['publishedAt'],
                       Subscribers = response['items'][i]['statistics']['subscriberCount'],
                       Views = response['items'][i]['statistics']['viewCount'],
                       country = response['items'][i]['snippet'].get('country'),
                       Total_videos = response['items'][i]['statistics']['videoCount'])
                       
            all_data.append(data)
    
    return all_data

In [38]:
channel_statitics = get_channel_stats(youtube, channel_ids)
channel_data = pd.DataFrame(channel_statitics)

In [40]:
data_channel = pd.merge(channel_data, channel_scraping, how='inner',on='channel_id')
data_channel = data_channel.sort_values('count',ascending=False)
data_channel

,Channel_name,channel_id,Channel_date,Subscribers,Views,country,Total_videos,count
23,CloudKid,UCSa8IUd1uEjlREMa21I3ZPQ,2013-07-30T20:10:22Z,5330000,2977615481,US,2766,1432
8,ImagineDragonsVEVO,UCpx_k19S2vUutWUUM9qmXEg,2012-01-23T18:04:00Z,9520000,14935596878,None,228,401
49,Adam Jensen,UChRJQ2QyKsY1EbBqM8GzzRg,2011-10-07T00:27:55Z,118000,91247548,US,71,330
45,McSkyz,UC4OjGShlHwNNd6jQ23GVJ5A,2011-07-18T09:49:59Z,1750000,284878737,FR,289,234
36,twenty one pilots,UCBQZwaNPFfJ1gZ1fLZpAEGw,2009-09-05T05:57:18Z,13400000,5837797696,US,217,193
...,...,...,...,...,...,...,...,...
272,Geon,UCwthtFbQ_4tcJAOMkKXLcNQ,2023-07-21T16:33:16.791997Z,50900,172696539,PK,254,11
283,Anthony Crazy,UCwlLTQ2yiKu0GCwNK70-pww,2018-04-02T20:50:23Z,375000,174629941,FR,69,11
282,Laura Laune,UCb32WqbvB979c0rDifGZVvA,2013-08-24T13:27:03Z,293000,81071269,FR,91,11
252,P2 DBOXS,UCgO3YEcCq98JZVF672wX0lg,2023-03-25T19:36:09.997693Z,318000,352213985,BD,103,11


In [42]:
# cleanning datatype : numeric
numeric_cols = ['Subscribers','Views','Total_videos']
data_channel[numeric_cols] = data_channel[numeric_cols].apply(pd.to_numeric, errors='coerce')

#cleaning datatype: datetime
data_channel['Channel_date'] = data_channel['Channel_date'].str.replace('T',' ').str.split('.').str[0]
data_channel['Channel_date'] = data_channel['Channel_date'].str.replace('Z','')
data_channel['Channel_date'] = pd.to_datetime(data_channel['Channel_date'])


#result
data_channel.dtypes

Channel_name            object
channel_id              object
Channel_date    datetime64[ns]
Subscribers              int64
Views                    int64
country                 object
Total_videos             int64
count                    int64
dtype: object

In [44]:
data_channel.head(3)

,Channel_name,channel_id,Channel_date,Subscribers,Views,country,Total_videos,count
23,CloudKid,UCSa8IUd1uEjlREMa21I3ZPQ,2013-07-30 20:10:22,5330000,2977615481,US,2766,1432
8,ImagineDragonsVEVO,UCpx_k19S2vUutWUUM9qmXEg,2012-01-23 18:04:00,9520000,14935596878,None,228,401
49,Adam Jensen,UChRJQ2QyKsY1EbBqM8GzzRg,2011-10-07 00:27:55,118000,91247548,US,71,330


<h2 align = center> Recuperation datas from all my videos history </h2>

### Recuperation data youtube's API

In [46]:
count_video = wt.video_ID.value_counts()
len(count_video)

16648

In [48]:
video_scraping = pd.DataFrame(data=count_video)
video_scraping = video_scraping.reset_index()
video_scraping.head(3)

,video_ID,count
0,EgwQG3MYp3o,67
1,i4AXVFggAnk,64
2,L3wKzyIN1yk,63


In [50]:
video_ids = video_scraping['video_ID'].tolist()

In [52]:
def get_video_details(youtube, video_ids):
    all_videos_stats = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part = 'snippet, statistics',
            id=','.join(video_ids[i:i+50]))
        response = request.execute()

        for video in response['items']:
            video_stats = dict(title = video['snippet']['title'],
                                Published_date = video['snippet']['publishedAt'],
                                Views = video['statistics'].get('viewCount'),
                                Like = video['statistics'].get('likeCount'),
                                Comments = video['statistics'].get('commentCount'),
                                Videocategory = video['snippet']['categoryId']
                              )
            all_videos_stats.append(video_stats)

    return all_videos_stats

In [54]:
video_statitics = get_video_details(youtube, video_ids)
video_data = pd.DataFrame(video_statitics)
video_data.head(3)

,title,Published_date,Views,Like,Comments,Videocategory
0,Silent Child - F**k You (Lyric Video),2020-08-26T15:27:16Z,42632575,600536,3379,10
1,Stela Cole - Love Like Mine,2020-09-25T15:27:08Z,17914064,140359,933,10
2,Rag'n'Bone Man - Human (Official Video),2016-07-21T11:00:00Z,1889820000,12239485,248434,10


In [56]:
data_video = pd.merge(video_data, wt, how='inner',on='title')
data_video.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22182 entries, 0 to 22181
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           22182 non-null  object        
 1   Published_date  22182 non-null  object        
 2   Views           22182 non-null  object        
 3   Like            21913 non-null  object        
 4   Comments        22023 non-null  object        
 5   Videocategory   22182 non-null  object        
 6   titleUrl        22182 non-null  object        
 7   time            22182 non-null  datetime64[ns]
 8   video_ID        22182 non-null  object        
 9   creator         22182 non-null  object        
 10  channel_url     22182 non-null  object        
 11  channel_id      22182 non-null  object        
dtypes: datetime64[ns](1), object(11)
memory usage: 2.0+ MB


### formating dataframe

In [58]:
# cleanning datatype : numeric
numeric_cols = ['Views','Like','Comments']
data_video[numeric_cols] = data_video[numeric_cols].apply(pd.to_numeric, errors='coerce')

#cleaning datatype: datetime
data_video['Published_date'] = data_video['Published_date'].str.split('T').str[0]
data_video['Published_date'] = pd.to_datetime(data_video['Published_date'])

#result
data_video.dtypes

title                     object
Published_date    datetime64[ns]
Views                      int64
Like                     float64
Comments                 float64
Videocategory             object
titleUrl                  object
time              datetime64[ns]
video_ID                  object
creator                   object
channel_url               object
channel_id                object
dtype: object

In [60]:
data_video.Videocategory.value_counts()

Videocategory
10    6143
24    5237
22    4156
23    1890
20    1193
27    1001
1      851
26     557
28     356
25     345
17     229
19     125
15      53
2       26
29      13
44       7
Name: count, dtype: int64

In [62]:
my_categories = {'10':'Music','24':'Entertainment','22':'People&Blog','23':'Comedy','20':'Gaming','27':'Education',
                 '1':'Film&Animation','26':'HowTo&Style','28':'Science&Technology','25':'News&Politics','17':'Sports',
                '19':'Travel&Events','15':'Pets&Animal','2':'Autos','29':'NonProfits&Activism','44':'Trailers'}

In [64]:
data_video['Videocategory'] = data_video['Videocategory'].map(my_categories)

In [66]:
data_video.Videocategory.value_counts()

Videocategory
Music                  6143
Entertainment          5237
People&Blog            4156
Comedy                 1890
Gaming                 1193
Education              1001
Film&Animation          851
HowTo&Style             557
Science&Technology      356
News&Politics           345
Sports                  229
Travel&Events           125
Pets&Animal              53
Autos                    26
NonProfits&Activism      13
Trailers                  7
Name: count, dtype: int64

<h2 align = center> Filter from 2018 to 2023 </h2>

In [68]:
data_video.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22182 entries, 0 to 22181
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           22182 non-null  object        
 1   Published_date  22182 non-null  datetime64[ns]
 2   Views           22182 non-null  int64         
 3   Like            21913 non-null  float64       
 4   Comments        22023 non-null  float64       
 5   Videocategory   22182 non-null  object        
 6   titleUrl        22182 non-null  object        
 7   time            22182 non-null  datetime64[ns]
 8   video_ID        22182 non-null  object        
 9   creator         22182 non-null  object        
 10  channel_url     22182 non-null  object        
 11  channel_id      22182 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(1), object(7)
memory usage: 2.0+ MB


In [70]:
data_video['year'] = data_video['time'].dt.year

nb_video_2017 = len(data_video[(data_video['year']<=2017)])
nb_video_2018 = len(data_video[(data_video['year']>2017)])

print(f'nb videos 2012 to 2017 : {nb_video_2017}')
print(f'nb videos 2018 to 2024 : {nb_video_2018}')

nb videos 2012 to 2017 : 1139
nb videos 2018 to 2024 : 21043


In [72]:
# filtrer le plus de données, enlevé 2024, data sur année entière
data_video_2018 = data_video.copy()
data_video_2018 = data_video_2018[(data_video_2018['year']>2017) & (data_video_2018['year']<2024)]
data_video_2018.groupby('year').size()

year
2018     1723
2019      365
2020      610
2021     1245
2022     2693
2023    11883
dtype: int64

<h2 align = center> Add the lenght of the videos </h2>

In [80]:
#excel transitoire pour récolter les durées manuellement (irrécupérable via API ou Selene) +suppr videos, watch only the begining ... 
df_video_id= pd.DataFrame(data_video_2018[['title','titleUrl','video_ID','year']])
df_video_id.to_excel('vd_id_time.xlsx')

In [82]:
#retrieve form excel avec durée en minute
df_duration = pd.read_excel('vd_id_time_duration.xlsx')
df_duration= df_duration.drop(["Colonne1"],axis=1)
df_duration

,title,titleUrl,video_ID,year,duration
0,Silent Child - F**k You (Lyric Video),https://www.youtube.com/watch?v=EgwQG3MYp3o,EgwQG3MYp3o,2023,3.30
1,Silent Child - F**k You (Lyric Video),https://www.youtube.com/watch?v=EgwQG3MYp3o,EgwQG3MYp3o,2023,3.30
2,Silent Child - F**k You (Lyric Video),https://www.youtube.com/watch?v=EgwQG3MYp3o,EgwQG3MYp3o,2023,3.30
3,Silent Child - F**k You (Lyric Video),https://www.youtube.com/watch?v=EgwQG3MYp3o,EgwQG3MYp3o,2023,3.30
4,Silent Child - F**k You (Lyric Video),https://www.youtube.com/watch?v=EgwQG3MYp3o,EgwQG3MYp3o,2023,3.30
...,...,...,...,...,...
18626,imagine the day they see you again..,https://www.youtube.com/watch?v=ZrBZNonf_qw,ZrBZNonf_qw,2023,0.07
18627,Je suis une BDH ?! 🚨,https://www.youtube.com/watch?v=hIlfuwJ9Of0,hIlfuwJ9Of0,2023,0.22
18628,Aww this was so cute 🥹❤️,https://www.youtube.com/watch?v=i-bZBXNExik,i-bZBXNExik,2023,0.27
18629,IM JUST KEN #Shorts,https://www.youtube.com/watch?v=947RyXTZ8Yo,947RyXTZ8Yo,2023,0.14


In [84]:
df_duration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18631 entries, 0 to 18630
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     18631 non-null  object 
 1   titleUrl  18631 non-null  object 
 2   video_ID  18631 non-null  object 
 3   year      18631 non-null  int64  
 4   duration  18631 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 727.9+ KB


In [102]:
# merge df_duration et raw_data
video_data_final = pd.merge(data_video_2018,df_duration,how='inner',on=['video_ID','title','titleUrl','year'])
video_data_final.head(3)

,title,Published_date,Views,Like,Comments,Videocategory,titleUrl,time,video_ID,creator,channel_url,channel_id,year,duration
0,Silent Child - F**k You (Lyric Video),2020-08-26,42632575,600536.0,3379.0,Music,https://www.youtube.com/watch?v=EgwQG3MYp3o,2023-09-13 18:40:34,EgwQG3MYp3o,CloudKid,https://www.youtube.com/channel/UCSa8IUd1uEjlR...,UCSa8IUd1uEjlREMa21I3ZPQ,2023,3.3
1,Silent Child - F**k You (Lyric Video),2020-08-26,42632575,600536.0,3379.0,Music,https://www.youtube.com/watch?v=EgwQG3MYp3o,2023-09-13 18:40:34,EgwQG3MYp3o,CloudKid,https://www.youtube.com/channel/UCSa8IUd1uEjlR...,UCSa8IUd1uEjlREMa21I3ZPQ,2023,3.3
2,Silent Child - F**k You (Lyric Video),2020-08-26,42632575,600536.0,3379.0,Music,https://www.youtube.com/watch?v=EgwQG3MYp3o,2023-09-13 18:40:34,EgwQG3MYp3o,CloudKid,https://www.youtube.com/channel/UCSa8IUd1uEjlR...,UCSa8IUd1uEjlREMa21I3ZPQ,2023,3.3


In [104]:
video_data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           68206 non-null  object        
 1   Published_date  68206 non-null  datetime64[ns]
 2   Views           68206 non-null  int64         
 3   Like            67947 non-null  float64       
 4   Comments        68067 non-null  float64       
 5   Videocategory   68206 non-null  object        
 6   titleUrl        68206 non-null  object        
 7   time            68206 non-null  datetime64[ns]
 8   video_ID        68206 non-null  object        
 9   creator         68206 non-null  object        
 10  channel_url     68206 non-null  object        
 11  channel_id      68206 non-null  object        
 12  year            68206 non-null  int32         
 13  duration        68206 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int32(1), int64(1), 

<h2 align = center> Fill the NULL values </h2>

In [106]:
video_data_final.isnull().sum()

title               0
Published_date      0
Views               0
Like              259
Comments          139
Videocategory       0
titleUrl            0
time                0
video_ID            0
creator             0
channel_url         0
channel_id          0
year                0
duration            0
dtype: int64

In [110]:
# corresponds to like=0 or comments=0
video_data_final['Like'] = video_data_final['Like'].fillna(0)
video_data_final['Comments'] = video_data_final['Comments'].fillna(0)
video_data_final.isnull().sum()

title             0
Published_date    0
Views             0
Like              0
Comments          0
Videocategory     0
titleUrl          0
time              0
video_ID          0
creator           0
channel_url       0
channel_id        0
year              0
duration          0
dtype: int64

<h2 align='center'> Save Data to Excel </h2>

In [121]:
with pd.ExcelWriter ("youtube_data.xlsx") as writer :
    #wt.to_excel(writer, sheet_name='working_data',index=False)
    tableau_pub.to_excel(writer, sheet_name='pub_data', index=False)
    data_channel.to_excel(writer, sheet_name='channel_data',index=False)
    data_video.to_excel(writer,sheet_name='video_data',index=False)
    #data_video_2018.to_excel(writer,sheet_name='video_data_filter_2018',index=False)
    video_data_final.to_excel(writer,sheet_name='video_data_complete',index=False)